In [1]:
# Import the required packages
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.utils import resample
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 20000)
pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings("ignore")
import datetime

<ipython-input-1-1e9fd325078d>:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
def flot_to_str(obj):
    """
    Convert Upc code from float to string.
    Use this function by applying lambda
    Parameters: "Upc" column of DataFrame
    Return:string converted Upc removing dot
    """
    while obj != 'np.nan':
        obj = str(obj).split('.')[0]
        if len(obj) == 10:
            obj = obj + '0'
        elif len(obj) == 4:
            obj = obj + '0000000' 
        return obj

In [3]:
def company(upcData):
    """
    Return company code from given Upc code.
    Parameters:'Upc' column of DataFrame
    Return: company code
    """
    try:
        code = upcData[: 6]
        if code == '000000':
            return x[-5]
        return code
    except:
        return -9999


In [4]:
def prodct(upcData):
    """
    Return company code from given Upc code.
    Parameters:'Upc' column of DataFrame
    Return: company code
    """
    try:
        code = upcData[6 :]
        return code
    except:
        return -9999

In [29]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # the next three operations are the ones we have just presented in the previous lines
    
    # drop the columns we won't use (it may be good to use them somehow)
    df = df.drop(["Upc", "FinelineNumber"], axis=1)

    # one-hot encoding for the DepartmentDescription
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)

    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)

    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [30]:
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble

clf = ensemble.RandomForestClassifier(random_state=2)
clf.fit(X_train, y_train);
#rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
#predictions = clf.predict(X_train)
from sklearn.metrics import classification_report

print(classification_report(y_valid, clf.predict(X_valid)))

#param_grid = { "criterion" : ["gini", "entropy"]}

#tree_clf = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

#tree_clf.fit(X_train, y_train)
#best_tree_clf = tree_clf.best_estimator_


RandomForestClassifier(random_state=2)

              precision    recall  f1-score   support

           3       0.78      0.81      0.79       794
           4       0.19      0.15      0.17        67
           5       0.71      0.77      0.74       976
           6       0.71      0.63      0.67       260
           7       0.62      0.65      0.63      1205
           8       0.74      0.80      0.77      2540
           9       0.70      0.68      0.69      1968
          12       0.29      0.04      0.07        49
          14       0.00      0.00      0.00         1
          15       0.52      0.35      0.42       204
          18       0.45      0.33      0.38       121
          19       0.35      0.23      0.27        75
          20       0.60      0.60      0.60       130
          21       0.62      0.62      0.62       155
          22       0.45      0.32      0.37       206
          23       0.36      0.31      0.33        26
          24       0.58      0.48      0.52       581
          25       0.64    

In [24]:
results = pd.DataFrame(columns=('clf', 'best_acc'))
print('Best Random Forest accuracy: ', tree_clf.best_score_)
print(best_tree_clf)
results = results.append({'clf': best_tree_clf, 'best_acc': tree_clf.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

Best Random Forest accuracy:  0.6807971014492754
RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=1)
The best classifier so far is: 
RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=1)


In [25]:
X.shape, XX.shape

((67029, 230), (28645, 230))

In [33]:
#yy = results.clf.iloc[0].predict(XX)
yy = clf.predict(XX)

In [34]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [35]:
submission.to_csv("randomForest_baseline.csv", header=True, index=False)